In [1]:
import pandas as pd
import numpy as np
import genra
import matplotlib.pyplot as plt
%matplotlib inline

from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes

from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs

In [2]:
import openpyxl

In [3]:
import seaborn as sns

#### Tutorial to show how to make GenRA predictions manually and reproduce the output with genra-py and compare with the value shown on the GenRA webapp

Manually making GenRA predictions of CHR:brain for Bisphenol A on the basis of Morgan chemical fingerprints

Extract neighbours from the GenRA webapp where there are results for this endpoint

In [8]:
bpa_df = {'smiles' : ['CC(C)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1', 'OC1=CC=C(O)C=C1', 'CC(=O)NC1=CC=C(O)C=C1', 'CC(C)(C)C1=CC(O)=CC=C1O', 'OC1=CC=CC=C1'],
          
          'dtxsid' :['DTXSID7020182', 'DTXSID7020716', 'DTXSID2020006', 'DTXSID6020220', 'DTXSID5021124'], 'y': [np.nan, 1,0,0,0], 'sim': [1, 0.4090909091, 0.2777777778, 0.2777777778, 0.2592592593]}



Convert into a dataframe. First row corresponds to Bisphenol A. Assume no data for this endpoint.

In [10]:
df = pd.DataFrame(bpa_df)

In [11]:
df

,smiles,dtxsid,y,sim
0,CC(C)(C1=CC=C(O)C=C1)C1=CC=C(O)C=C1,DTXSID7020182,NaN,1.000000
1,OC1=CC=C(O)C=C1,DTXSID7020716,1.0,0.409091
2,CC(=O)NC1=CC=C(O)C=C1,DTXSID2020006,0.0,0.277778
3,CC(C)(C)C1=CC(O)=CC=C1O,DTXSID6020220,0.0,0.277778
4,OC1=CC=CC=C1,DTXSID5021124,0.0,0.259259


Use smiles to create Morgan FP representations

In [12]:
MOLS = dict(zip(df['dtxsid'], df['smiles']))
MOLS = {k:Chem.MolFromSmiles(v) for k,v in MOLS.items()}
MOLS = {i:j for i,j in MOLS.items() if j}
FP0 = pd.DataFrame([np.array(AllChem.GetMorganFingerprintAsBitVect(i,3,2048)) for i in MOLS.values()])
FP0.index=MOLS.keys()
FP0.columns = ['mrgn_%d'%i for i in FP0.columns]

Create a FP dataframe for Bisphenol A and its neighbours. Morgan fingerprint settings are a radius of 3 and a bitvector length of 2048.

In [14]:
FP0

,mrgn_0,mrgn_1,mrgn_2,mrgn_3,mrgn_4,mrgn_5,mrgn_6,mrgn_7,mrgn_8,mrgn_9,...,mrgn_2038,mrgn_2039,mrgn_2040,mrgn_2041,mrgn_2042,mrgn_2043,mrgn_2044,mrgn_2045,mrgn_2046,mrgn_2047
DTXSID7020182,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID7020716,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID2020006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID6020220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DTXSID5021124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create a distance matrix in order to retrieve the pairwise similarities of Bisphenol A relative to its neighbours.

In [15]:
from scipy.spatial.distance import pdist, squareform

In [16]:
def distance_matrix(df):
    '''
    Function to create a pairwise square distance matrix using the Jaccard index
    '''
    D_mgrn = pd.DataFrame(squareform(pdist(df, 'jaccard')), columns = df.index, index = df.index)

    return D_mgrn

In [17]:
S = 1 -  distance_matrix(FP0)

Pairwise similarities of BPA and its neighbours are in the first column of similarity matrix S

In [22]:
S.iloc[1:,0].values

array([0.40909091, 0.27777778, 0.27777778, 0.25925926])

Activities are the actual experimental results captured in column y of the original df

In [25]:
df.iloc[1:, 2].values

array([1., 0., 0., 0.])

Similarity weighted activity score is the sum of the pairwise similarities * activities divided by the sum of the pairwise similarities

In [35]:
SWA = np.sum((S.iloc[1:,0].values * df.iloc[1:, 2].values))/np.sum(S.iloc[1:,0].values)

In [42]:
print('The similarity weighted activity of Bisphenol A is {}'.format(round(SWA,3)))

The similarity weighted activity of Bisphenol A is 0.334


This corresponds to our GenRA prediction

Use the GenRA webapp (comptox.epa.gov/genra) to make a prediction of CHR:brain for Bisphenol A on the basis of Morgan chemical fingerprints

ACT score reported using genra-py engine within the GenRA webapp gives rise to a score of 0.334. Check - manual calculation corresponds to what is presented in the webapp.

Using genra-py to produce the same prediction.

In [69]:
from genra.rax.skl.cls import GenRAPredClass 

Fit GenRA model using  X (fingerprints for the neighbours of Bisphenol A) and y  (the data values for those neighbours)

In [70]:
X = FP0.iloc[1:,:]

In [71]:
y = df.iloc[1:,-2].values

Model is the GenRAPredClass. Need jaccard to make sure that the Jaccard similarity metric is used and weights ensures that the similarity weighted activities are returned

In [72]:
GP = GenRAPredClass(metric = 'jaccard', weights = lambda distances: 1-distances,n_neighbors = 4)

In [73]:
GP.fit(X.values, y)

GenRAPredClass(n_neighbors=4)

To run predictions need to take the FP for Bisphenol A and reshape it so it can be passed to the model class

Predict will give the final binary prediction of active/inactive for Bisphenol A. Predict_proba will give the probabilities for the prediction and we use logic to return the probability corresponding to the activity class predicted. In this case the maximum probability is associated 

In [74]:
pred = GP.predict(FP0.iloc[0,:].values.reshape(1,-1))[0]

/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [75]:
pred

0.0

In [76]:
proba = np.max(GP.predict_proba(FP0.iloc[0,:].values.reshape(1,-1)))

/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [77]:
proba

0.6657496561210454

- swa = proba if pred esle 1 - proba


In [78]:
swa = proba if pred else 1-proba
swa

0.3342503438789546

Toy example to show how to construct hybrid fingerprint representations

In [79]:
data = {'fp1_0' : [1,1,0,0,1,1], 'fp1_1' : [0,1,0,0,0,1], 'fp1_2' :[0,0,0,1,1,0], 'fp1_3':[0,1,1,1,0,0], 'fp1_4': [1,0,1,1,1,0], 'fp2_0' : [1,0,1,0,1,0], 'fp2_1' : [0,0,0,1,1,1], 'fp2_2': [1,1,0,1,0,0], 'fp3_0' : [1,0,0,1,0,0], 'fp3_1': [0,1,0,0,1,1], 'fp3_2': [0,1,1,1,1,0], 'fp3_3': [1,0,0,1,1,1]}

In [80]:
df1 = pd.DataFrame(data, index = ['target', 'c0','c1', 'c2', 'c3', 'c4'])

In [81]:
df1['Y'] = [np.nan, 1,0,1,0,1]

In [86]:
df1

,fp1_0,fp1_1,fp1_2,fp1_3,fp1_4,fp2_0,fp2_1,fp2_2,fp3_0,fp3_1,fp3_2,fp3_3,Y
target,1,0,0,0,1,1,0,1,1,0,0,1,NaN
c0,1,1,0,1,0,0,0,1,0,1,1,0,1.0
c1,0,0,0,1,1,1,0,0,0,0,1,0,0.0
c2,0,0,1,1,1,0,1,1,1,0,1,1,1.0
c3,1,0,1,0,1,1,1,0,0,1,1,1,0.0
c4,1,1,0,0,0,0,1,0,0,1,0,1,1.0


FP_1 are going to have a weight of 2, FP_2 a weight of 1 and FP_3 a weight of 3

First we create the distance matrices, convert them into similarity matrices and then normalise them.

In [90]:
S =1- distance_matrix(df1.iloc[:, 0:5])


In [93]:
S2 = S*2
S2 = S2.iloc[:,1:]
S2

,c0,c1,c2,c3,c4
target,0.500000,0.666667,0.500000,1.333333,0.666667
c0,2.000000,0.500000,0.400000,0.400000,1.333333
c1,0.500000,2.000000,1.333333,0.500000,0.000000
c2,0.400000,1.333333,2.000000,1.000000,0.000000
c3,0.400000,0.500000,1.000000,2.000000,0.500000
c4,1.333333,0.000000,0.000000,0.500000,2.000000


In [109]:
S1 = 1 - distance_matrix(df1.iloc[:, 5:8])
S1 = S1.iloc[:,1:]
S1

,c0,c1,c2,c3,c4
target,0.5,0.5,0.333333,0.333333,0.0
c0,1.0,0.0,0.500000,0.000000,0.0
c1,0.0,1.0,0.000000,0.500000,0.0
c2,0.5,0.0,1.000000,0.333333,0.5
c3,0.0,0.5,0.333333,1.000000,0.5
c4,0.0,0.0,0.500000,0.500000,1.0


In [120]:

S3 = 1 - distance_matrix(df1.iloc[:, 8:12])
S3 = S3.iloc[:, 1:]
S3 = 3 * S3
S3

,c0,c1,c2,c3,c4
target,0.00,0.0,2.00,0.75,1.00
c0,3.00,1.5,0.75,2.00,1.00
c1,1.50,3.0,1.00,1.00,0.00
c2,0.75,1.0,3.00,1.50,0.75
c3,2.00,1.0,1.50,3.00,2.00
c4,1.00,0.0,0.75,2.00,3.00


In [121]:
new_df = pd.DataFrame()
for column in S2.columns:
    weights = 6
    new_df[column] = S2[column] + S1[column] + S3[column]
new_df = new_df/6

new_df['Y'] = [np.nan, 1,0,1,0,1]

new_df['row_sum'] = new_df.sum(axis = 1)
new_df

,c0,c1,c2,c3,c4,Y,row_sum
target,0.166667,0.194444,0.472222,0.402778,0.277778,NaN,1.513889
c0,1.000000,0.333333,0.275000,0.400000,0.388889,1.0,3.397222
c1,0.333333,1.000000,0.388889,0.333333,0.000000,0.0,2.055556
c2,0.275000,0.388889,1.000000,0.472222,0.208333,1.0,3.344444
c3,0.400000,0.333333,0.472222,1.000000,0.500000,0.0,2.705556
c4,0.388889,0.000000,0.208333,0.500000,1.000000,1.0,3.097222


Now we have a similarity matrix where the weights are adjusted for each fingerprint type. Now we can calculate the SWA for the target chemical.

In [125]:
np.sum(new_df.iloc[0,:-2] * new_df.iloc[1:,-2])/new_df.iloc[0,-1]

0.6055045871559633

Replicating this in genra-py

In [123]:
X_train = df1.iloc[1:,:-1].values
y_train = df1.iloc[1:,-1].values

Capture the fingerprint types by slice notation

In [124]:
slices = [slice(0,5), slice(5,8), slice(8,None)]

Capture the weights by the hybrid weights argument

In [126]:
hybrid_weights = [2, 1, 3]

In [128]:
from genra.rax.skl.hybrid import GenRAPredBinaryHybrid

In [129]:
hybrid = GenRAPredBinaryHybrid(metric = 'jaccard', slices = slices, hybrid_weights = hybrid_weights, weights = lambda distances: 1-distances)

In [130]:
hybrid.fit(X_train, y_train)

GenRAPredBinaryHybrid(hybrid_weights=[2, 1, 3], metric='jaccard',
                      slices=[slice(0, 5, None), slice(5, 8, None),
                              slice(8, None, None)])

In [131]:
X_test = df1.iloc[0,:-1].values

In [132]:
proba = np.max(hybrid.predict_proba(X_test.reshape(1,-1)))

/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [133]:
pred = hybrid.predict(X_test.reshape(1,-1))[0]

/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/grace/Documents/python/misc/tutotial_env/lib/python3.9/site-packages/sklearn/metrics/pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [134]:
swa = proba if pred else 1-proba
swa

0.6055045871559633

Similarity weighted activities correspond with each other.